# Data preparation
## Read and show Excel data for one day from Flightradar

In [205]:
import pandas as pd

In [206]:
# XLSX-Datei laden
flightradar = pd.read_excel("raw/250518_flightradar.xlsx")

# Erste 5 Zeilen anzeigen
print(flightradar.head(5))  

  web-scraper-order                              web-scraper-start-url  page  \
0      1747631713-1  https://www.flightradar24.com/data/airports/zr...   NaN   
1      1747631713-2  https://www.flightradar24.com/data/airports/zr...   NaN   
2      1747631713-3  https://www.flightradar24.com/data/airports/zr...   NaN   
3      1747631713-4  https://www.flightradar24.com/data/airports/zr...   NaN   
4      1747631713-5  https://www.flightradar24.com/data/airports/zr...   NaN   

   page2  page3  page4  page5  page6  page7      TIME  FLIGHT  \
0    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00   CS638   
1    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00   CS600   
2    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00  W22160   
3    NaN    NaN    NaN    NaN    NaN    NaN  05:45:00  W22178   
4    NaN    NaN    NaN    NaN    NaN    NaN  06:00:00   WK130   

       DESTINATION           AIRLINE       AIRCRAFT          STATUS  
0     Ohrid (OHD)-  Chair Airlines -  A320 (HB-JOP)  Depar

## Show data types and shape of the DataFrame

In [207]:
print(flightradar.dtypes)
print(flightradar.shape)  

web-scraper-order         object
web-scraper-start-url     object
page                     float64
page2                    float64
page3                    float64
page4                    float64
page5                    float64
page6                    float64
page7                    float64
TIME                      object
FLIGHT                    object
DESTINATION               object
AIRLINE                   object
AIRCRAFT                  object
STATUS                    object
dtype: object
(387, 15)


## The Date is missing in the DataFrame. Create a new field DATETIME
### DATETIME is the planned departure time for that day

In [208]:
# Datum als Variable definieren
date_str = "2025-05-18"

In [209]:
# TIME als String sicherstellen (falls es z. B. float ist)
flightradar["TIME"] = flightradar["TIME"].astype(str)

# DATETIME-Feld erzeugen: 'date_str' + Zeit aus TIME-Feld
flightradar["DATETIME"] = date_str + " " + flightradar["TIME"]

# Kontrolle
print(flightradar[["TIME", "DATETIME"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

        TIME             DATETIME
0   05:45:00  2025-05-18 05:45:00
1   05:45:00  2025-05-18 05:45:00
2   05:45:00  2025-05-18 05:45:00
3   05:45:00  2025-05-18 05:45:00
4   06:00:00  2025-05-18 06:00:00
5   06:00:00  2025-05-18 06:00:00
6   06:10:00  2025-05-18 06:10:00
7   06:10:00  2025-05-18 06:10:00
8   06:20:00  2025-05-18 06:20:00
9   06:20:00  2025-05-18 06:20:00
10  06:25:00  2025-05-18 06:25:00
11  06:25:00  2025-05-18 06:25:00
12  06:25:00  2025-05-18 06:25:00
13  06:30:00  2025-05-18 06:30:00
14  06:30:00  2025-05-18 06:30:00
15  06:35:00  2025-05-18 06:35:00
16  06:35:00  2025-05-18 06:35:00
17  06:40:00  2025-05-18 06:40:00
18  06:45:00  2025-05-18 06:45:00
19  06:45:00  2025-05-18 06:45:00
20  06:45:00  2025-05-18 06:45:00
21  06:45:00  2025-05-18 06:45:00
22  06:50:00  2025-05-18 06:50:00
23  06:50:00  2025-05-18 06:50:00
24  06:50:00  2025-05-18 06:50:00
25  06:55:00  2025-05-18 06:55:00
26  06:55:00  2025-05-18 06:55:00
27  06:55:00  2025-05-18 06:55:00
28  07:00:00  

## Create two new fields: DESTINATION_CLEAN and IATA_CODE using regex

In [210]:
# DESTINATION_CLEAN: Alles vor der Klammer
flightradar["DESTINATION_CLEAN"] = flightradar["DESTINATION"].str.extract(r"^(.*?)\s*\(")

# IATA-CODE: Inhalt in der Klammer
flightradar["IATA_CODE"] = flightradar["DESTINATION"].str.extract(r"\((\w{3})\)")

# Kontrolle
print(flightradar[["DESTINATION", "DESTINATION_CLEAN", "IATA_CODE"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

                 DESTINATION  DESTINATION_CLEAN IATA_CODE
0               Ohrid (OHD)-              Ohrid       OHD
1            Pristina (PRN)-           Pristina       PRN
2            Pristina (PRN)-           Pristina       PRN
3               Ohrid (OHD)-              Ohrid       OHD
4            Hurghada (HRG)-           Hurghada       HRG
5             Antalya (AYT)-            Antalya       AYT
6              Athens (ATH)-             Athens       ATH
7              Lisbon (LIS)-             Lisbon       LIS
8               Porto (OPO)-              Porto       OPO
9            Tenerife (TFS)-           Tenerife       TFS
10       Gran Canaria (LPA)-       Gran Canaria       LPA
11         Marsa Alam (RMF)-         Marsa Alam       RMF
12             Bilbao (BIO)-             Bilbao       BIO
13      Fuerteventura (FUE)-      Fuerteventura       FUE
14          Lanzarote (ACE)-          Lanzarote       ACE
15             Dublin (DUB)-             Dublin       DUB
16            

## Create three new fields: CANCELED, DEPART_TIME and DEPART_DATETIME using regex

In [211]:
# 1. CANCELED-Spalte (Boolean)
flightradar["CANCELED"] = flightradar["STATUS"].str.contains("Canceled", na=False)

# 2. DEPART_TIME-Spalte (nur wenn "Departed HH:MM" enthalten ist)
flightradar["DEPART_TIME"] = flightradar["STATUS"].str.extract(r"Departed\s*(\d{2}:\d{2})")[0]

# 3. DEPART_DATETIME-Spalte: date_str + DEPART_TIME (fallback "00:00") + ":00"
flightradar["DEPART_DATETIME"] = (
    date_str
    + " "
    + flightradar["DEPART_TIME"].fillna("00:00") # falls keine Zeit da ist, setze "00:00"
    + ":00" # Sekunden anhängen
)

# Kontrolle
print(flightradar[["STATUS","CANCELED","DEPART_TIME","DEPART_DATETIME"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

                   STATUS  CANCELED DEPART_TIME      DEPART_DATETIME
0          Departed 06:00     False       06:00  2025-05-18 06:00:00
1          Departed 06:02     False       06:02  2025-05-18 06:02:00
2                 Unknown     False         NaN  2025-05-18 00:00:00
3                 Unknown     False         NaN  2025-05-18 00:00:00
4          Departed 06:09     False       06:09  2025-05-18 06:09:00
5          Departed 06:11     False       06:11  2025-05-18 06:11:00
6          Departed 06:53     False       06:53  2025-05-18 06:53:00
7          Departed 06:21     False       06:21  2025-05-18 06:21:00
8          Departed 06:49     False       06:49  2025-05-18 06:49:00
9          Departed 06:47     False       06:47  2025-05-18 06:47:00
10         Departed 06:39     False       06:39  2025-05-18 06:39:00
11         Departed 06:41     False       06:41  2025-05-18 06:41:00
12         Departed 06:51     False       06:51  2025-05-18 06:51:00
13         Departed 06:43     Fals

## Creating the AIRLINE_CLEAN field and removing unimportant parts

In [212]:
flightradar["AIRLINE_CLEAN"] = (
    flightradar["AIRLINE"]
      .str.extract(r'^(.+?)(?=\s*(?:\(|-))')[0]  # Gruppe 1: alles bis vor "(" oder "-"
      .str.strip()                               # führende/trailing Leerzeichen entfernen
)

# Kontrolle
print(flightradar[["AIRLINE","AIRLINE_CLEAN"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

                                   AIRLINE     AIRLINE_CLEAN
0                         Chair Airlines -    Chair Airlines
1                         Chair Airlines -    Chair Airlines
2                             Flexflight -        Flexflight
3                             Flexflight -        Flexflight
4                          Edelweiss Air -     Edelweiss Air
5                          Edelweiss Air -     Edelweiss Air
6                       Helvetic Airways -  Helvetic Airways
7                       TAP Air Portugal -  TAP Air Portugal
8                                  Swiss -             Swiss
9                          Edelweiss Air -     Edelweiss Air
10                         Edelweiss Air -     Edelweiss Air
11                         Edelweiss Air -     Edelweiss Air
12                                 Swiss -             Swiss
13                         Edelweiss Air -     Edelweiss Air
14  Edelweiss Air (Help Alliance Livery) -     Edelweiss Air
15                      

## Creating the AIRCRAFT_SHORT field

In [213]:
# Create the AIRCRAFT_SHORT field by removing text inside parentheses
flightradar["AIRCRAFT_SHORT"] = (
    flightradar["AIRCRAFT"]
      .str.replace(r"\s*\(.*?\)", "", regex=True)  # non‑greedy, regex=True
      .str.strip()
)

# Check the result
print(flightradar[["AIRCRAFT", "AIRCRAFT_SHORT"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

         AIRCRAFT AIRCRAFT_SHORT
0   A320 (HB-JOP)           A320
1   A320 (HB-JOS)           A320
2          320 ()            320
3          320 ()            320
4   A320 (HB-IHX)           A320
5   A320 (HB-JLS)           A320
6   E295 (HB-AZI)           E295
7   A20N (CS-TVA)           A20N
8   BCS3 (HB-JCF)           BCS3
9   A320 (HB-IHY)           A320
10  A320 (HB-IJU)           A320
11  A320 (HB-JJM)           A320
12  A320 (HB-IJI)           A320
13  A320 (HB-JJN)           A320
14  A320 (HB-JLT)           A320
15  BCS3 (HB-JCD)           BCS3
16  A320 (HB-JLP)           A320
17  A320 (HB-IJV)           A320
18  A319 (HB-JOJ)           A319
19  A320 (HB-JLR)           A320
20  A320 (HB-IJQ)           A320
21         320 ()            320
22  E290 (HB-AZG)           E290
23  E190 (HB-JVY)           E190
24  A21N (HB-JPA)           A21N
25  E295 (HB-AZJ)           E295
26  A321 (HB-IOD)           A321
27  E295 (HB-AZL)           E295
28  B738 (PH-BXZ)           B738
29  BCS1 (

## Create a new field DELAY and DELAY_MINUTES and calculate it from DATETIME and DEPART_DATETIME

In [214]:
# 1) Stelle sicher, dass beide Spalten als datetime64 vorliegen
flightradar["DATETIME"] = pd.to_datetime(flightradar["DATETIME"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
flightradar["DEPART_DATETIME"] = pd.to_datetime(flightradar["DEPART_DATETIME"], format="%Y-%m-%d %H:%M:%S", errors="coerce")

# 2) DELAY als Differenz berechnen
flightradar["DELAY"] = flightradar["DEPART_DATETIME"] - flightradar["DATETIME"]

# 3a) Variante A: Floor‑Division auf total_seconds, dann in int
flightradar["DELAY_MINUTES"] = (
    flightradar["DELAY"].dt.total_seconds() // 60
).astype(int)

# Kontrolle
print(flightradar[["DATETIME","DEPART_DATETIME","DELAY","DELAY_MINUTES"]].head(50))
print("\n")
print(flightradar.dtypes)
print(flightradar.shape)

              DATETIME     DEPART_DATETIME             DELAY  DELAY_MINUTES
0  2025-05-18 05:45:00 2025-05-18 06:00:00   0 days 00:15:00             15
1  2025-05-18 05:45:00 2025-05-18 06:02:00   0 days 00:17:00             17
2  2025-05-18 05:45:00 2025-05-18 00:00:00 -1 days +18:15:00           -345
3  2025-05-18 05:45:00 2025-05-18 00:00:00 -1 days +18:15:00           -345
4  2025-05-18 06:00:00 2025-05-18 06:09:00   0 days 00:09:00              9
5  2025-05-18 06:00:00 2025-05-18 06:11:00   0 days 00:11:00             11
6  2025-05-18 06:10:00 2025-05-18 06:53:00   0 days 00:43:00             43
7  2025-05-18 06:10:00 2025-05-18 06:21:00   0 days 00:11:00             11
8  2025-05-18 06:20:00 2025-05-18 06:49:00   0 days 00:29:00             29
9  2025-05-18 06:20:00 2025-05-18 06:47:00   0 days 00:27:00             27
10 2025-05-18 06:25:00 2025-05-18 06:39:00   0 days 00:14:00             14
11 2025-05-18 06:25:00 2025-05-18 06:41:00   0 days 00:16:00             16
12 2025-05-1

## Showing full DataFrame

In [215]:
flightradar.head(3)

,web-scraper-order,web-scraper-start-url,page,page2,page3,page4,page5,page6,page7,TIME,...,DATETIME,DESTINATION_CLEAN,IATA_CODE,CANCELED,DEPART_TIME,DEPART_DATETIME,AIRLINE_CLEAN,AIRCRAFT_SHORT,DELAY,DELAY_MINUTES
0,1747631713-1,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-18 05:45:00,Ohrid,OHD,False,06:00,2025-05-18 06:00:00,Chair Airlines,A320,0 days 00:15:00,15
1,1747631713-2,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-18 05:45:00,Pristina,PRN,False,06:02,2025-05-18 06:02:00,Chair Airlines,A320,0 days 00:17:00,17
2,1747631713-3,https://www.flightradar24.com/data/airports/zr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05:45:00,...,2025-05-18 05:45:00,Pristina,PRN,False,NaN,2025-05-18 00:00:00,Flexflight,320,-1 days +18:15:00,-345


## Creating a New DataFrame with Selected Columns

In [216]:
# Create a new DataFrame with selected columns
flightradar_clean = flightradar[[
    "DESTINATION_CLEAN", 
    "IATA_CODE", 
    "AIRLINE_CLEAN", 
    "AIRCRAFT_SHORT", 
    "CANCELED", 
    "DATETIME",
    "DEPART_TIME", 
    "DEPART_DATETIME", 
    "DELAY_MINUTES"
]].copy()

# Check the result
print(flightradar_clean.head(50))
print("\n")
print(flightradar_clean.dtypes)
print(flightradar_clean.shape)

    DESTINATION_CLEAN IATA_CODE     AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
0               Ohrid       OHD    Chair Airlines           A320     False   
1            Pristina       PRN    Chair Airlines           A320     False   
2            Pristina       PRN        Flexflight            320     False   
3               Ohrid       OHD        Flexflight            320     False   
4            Hurghada       HRG     Edelweiss Air           A320     False   
5             Antalya       AYT     Edelweiss Air           A320     False   
6              Athens       ATH  Helvetic Airways           E295     False   
7              Lisbon       LIS  TAP Air Portugal           A20N     False   
8               Porto       OPO             Swiss           BCS3     False   
9            Tenerife       TFS     Edelweiss Air           A320     False   
10       Gran Canaria       LPA     Edelweiss Air           A320     False   
11         Marsa Alam       RMF     Edelweiss Air           A320

## Count missing values in columns

In [217]:
# Count missing values per column
print(flightradar_clean.isnull().sum())

# Total number of missing values in the DataFrame
print("Total missing values:", flightradar_clean.isnull().sum().sum())

DESTINATION_CLEAN     0
IATA_CODE             0
AIRLINE_CLEAN         7
AIRCRAFT_SHORT        0
CANCELED              0
DATETIME              0
DEPART_TIME          13
DEPART_DATETIME       0
DELAY_MINUTES         0
dtype: int64
Total missing values: 20


## Show rows with missing values in the DataFrame

In [218]:
# Filter rows with any missing values
missing_rows = flightradar_clean[flightradar_clean.isnull().any(axis=1)]

# Display rows with missing values
print(missing_rows)


     DESTINATION_CLEAN IATA_CODE    AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
2             Pristina       PRN       Flexflight            320     False   
3                Ohrid       OHD       Flexflight            320     False   
21              Skopje       SKP       Flexflight            320     False   
30             Palermo       PMO            Swiss           A20N     False   
38              London       LHR            Swiss           A21N     False   
55          Manchester       MAN              NaN           GL5T     False   
63            Pristina       PRN      GP Aviation            734     False   
81             Rostock       RLG     euroairlines            738     False   
123           Hurghada       HRG       Flexflight            320     False   
127  Palma de Mallorca       PMI       Flexflight            320     False   
128          Groningen       GRQ              NaN           SF50     False   
131               Rome       CIA              NaN           C25A

## Show the DataFrame without missing values

In [219]:
# Drop rows with any missing values (NaN)
flightradar_clean = flightradar_clean.dropna(axis=0)

# Display the DataFrame after dropping rows with missing values
print(flightradar_clean)


    DESTINATION_CLEAN IATA_CODE     AIRLINE_CLEAN AIRCRAFT_SHORT  CANCELED  \
0               Ohrid       OHD    Chair Airlines           A320     False   
1            Pristina       PRN    Chair Airlines           A320     False   
4            Hurghada       HRG     Edelweiss Air           A320     False   
5             Antalya       AYT     Edelweiss Air           A320     False   
6              Athens       ATH  Helvetic Airways           E295     False   
..                ...       ...               ...            ...       ...   
382         Sao Paulo       GRU             Swiss           B77W     False   
383         Singapore       SIN             Swiss           B77W     False   
384         Hong Kong       HKG             Swiss           B77W     False   
385             Milan       MXP  Helvetic Airways           E190     False   
386      Johannesburg       JNB             Swiss           A343     False   

               DATETIME DEPART_TIME     DEPART_DATETIME  DELAY_

## Check for duplicate rows in the DataFrame and display them if any exist

In [220]:
# Duplikate prüfen
duplicates = flightradar_clean[flightradar_clean.duplicated()]

# Duplikate anzeigen (falls vorhanden)
print(duplicates)

# Anzahl der Duplikate
print(f"Number of duplicate rows: {duplicates.shape[0]}")

Empty DataFrame
Columns: [DESTINATION_CLEAN, IATA_CODE, AIRLINE_CLEAN, AIRCRAFT_SHORT, CANCELED, DATETIME, DEPART_TIME, DEPART_DATETIME, DELAY_MINUTES]
Index: []
Number of duplicate rows: 0


## Save the cleaned DataFrame to a CSV file for further use or analysis

In [221]:
# Speichern des DataFrames als CSV-Datei
flightradar_clean.to_csv("prepared/250518_flightradar_prepared.csv", index=False)
